In [1]:
import duckdb

ModuleNotFoundError: No module named 'duckdb'

In [ ]:
!pip install library_name

In [2]:
!pip install duckdb

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.3 MB 330.3 kB/s eta 0:00:38
   ---------------------------------------- 0.1/12.3 MB 469.7 kB/s eta 0:00:27
   ---------------------------------------- 0.1/12.3 MB 590.8 kB/s eta 0:00:21
    --------------------------------------- 0.2/12.3 MB 958.4 kB/s eta 0:00:13
    --------------------------------------- 0.3/12.3 MB 1.1 MB/s eta 0:00:12
   - -------------------------------------- 0.4/12.3 MB 1.2 MB/s eta 0:00:10
   -- ------------------------------------- 0.7/12.3 MB 1.9 MB/s eta 0:00:07
   --- ------------------------------------ 1.0/12.3 MB 2.6 MB/s eta 0:00:05
   ---- ----------------------------------- 1.5/12.3 MB 3.5 MB/s eta 0:00:04
   ------ --------------------------------- 2.1/12.3 MB 4.3 MB/s eta 0:00:03
   -------- ------------------------------- 2.6/12.3 MB 4.9 MB/s eta 0:00:02
   --

In [2]:
import duckdb
import pandas as pd
import os
from datetime import datetime


In [3]:
# Estabelece conexão com o banco de dados DuckDB
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [16]:
# leitura dos arquivos CSV
arquivo = 'z0019_2.csv'
data_ingestao = datetime.now()
df = pd.read_csv(f'../landing/{arquivo}',sep=';')
df['nome_arquivo'] = arquivo
df['data_ingestao'] = data_ingestao
df

,NATBR,MAKTX,WEARKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10004,SERRA,BT50,100,200,z0019_2.csv,2026-01-19 20:34:25.981050
1,10005,MACHADO,BT50,100,100,z0019_2.csv,2026-01-19 20:34:25.981050
2,10003,PREGO,BT10,100,50,z0019_2.csv,2026-01-19 20:34:25.981050


In [17]:
# Criação da camada bronze

con.execute("""
        CREATE TABLE IF NOT EXISTS bronze_produtos (
            NATBR VARCHAR,
            MARTX VARCHAR,
            WERKS VARCHAR,
            MAINS VARCHAR,
            LABST VARCHAR,
            nome_arquivo VARCHAR,
            data_ingestao TIMESTAMP
        )
    """
)



In [18]:
# Ingestão da tabela bronze
con.execute("""
        INSERT INTO bronze_produtos 
        SELECT * FROM df
    """
)

In [21]:
# Consulta para verificar os dados inseridos
resultado = con.execute("SELECT * FROM bronze_z0019").fetchdf()
resultado.head()

,NATBR,MARTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-01-19 20:27:43.188722
1,10002,MARTELO,BT50,100,1500,z0019_1.csv,2026-01-19 20:27:43.188722
2,10003,PREGO,BT10,100,50,z0019_1.csv,2026-01-19 20:27:43.188722
3,10004,SERRA,BT50,100,200,z0019_2.csv,2026-01-19 20:34:25.981050
4,10005,MACHADO,BT50,100,100,z0019_2.csv,2026-01-19 20:34:25.981050


In [20]:
# Alterando ao nome da tabela para o nome bruto
con.execute("""
        ALTER TABLE bronze_produtos RENAME TO bronze_z0019
    """
)

In [22]:
# Finalizando a conexão com o banco de dados
con.close()